# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,zambezi,-13.5432,23.1047,24.39,57,93,1.92,ZM,1736356022
1,1,hermanus,-34.4187,19.2345,21.00,67,6,2.89,ZA,1736356023
2,2,saldanha,-33.0117,17.9442,23.03,76,0,6.04,ZA,1736355834
3,3,kodiak,57.7900,-152.4072,0.73,86,0,0.00,US,1736356026
4,4,metairie,29.9841,-90.1528,4.00,73,75,5.14,US,1736355732


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',        # Column for longitude
    'Lat',        # Column for latitude
    geo=True,     # Enable geospatial plotting
    tiles='OSM',  # Base map layer
    size=10,      # Size of the points
    color='blue', # Color of the points
    alpha=0.6     # Transparency
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
desired_city_ids = [
    43, 66, 102, 120, 130, 138, 146, 160, 170, 176, 
    289, 332, 337, 349, 399, 446, 460, 461, 465, 506
]
filtered_cities_df = city_data_df[city_data_df['City_ID'].isin(desired_city_ids)]

# Sort the filtered data by City_ID for consistency with the output format
filtered_cities_df = filtered_cities_df.sort_values(by='City_ID')

# Display the filtered DataFrame
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,vorgashor,67.5833,63.9500,-28.50,100,100,1.81,RU,1736356073
66,66,badger,64.8000,-147.5333,-11.68,88,75,0.00,US,1736356100
102,102,port mathurin,-19.6833,63.4167,26.26,87,3,10.14,MU,1736355611
120,120,mejit,10.2753,170.8646,27.00,74,16,11.09,MH,1736355635
130,130,tongren,27.7172,109.1853,5.17,91,100,1.47,CN,1736355646
138,138,raja,8.4596,25.6780,25.79,21,9,1.83,SS,1736355656
146,146,nuuk,64.1835,-51.7216,-3.66,63,40,1.03,GL,1736355665
160,160,margaret river,-33.9500,115.0667,12.65,84,100,3.35,AU,1736355685
170,170,derazhnya,49.2692,27.4338,6.14,88,100,6.26,UA,1736355702
176,176,vuktyl,63.8567,57.3094,-23.64,100,70,1.44,RU,1736355708


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
43,vorgashor,RU,67.5833,63.9500,100,
66,badger,US,64.8000,-147.5333,88,
102,port mathurin,MU,-19.6833,63.4167,87,
120,mejit,MH,10.2753,170.8646,74,
130,tongren,CN,27.7172,109.1853,91,
138,raja,SS,8.4596,25.6780,21,
146,nuuk,GL,64.1835,-51.7216,63,
160,margaret river,AU,-33.9500,115.0667,84,
170,derazhnya,UA,49.2692,27.4338,88,
176,vuktyl,RU,63.8567,57.3094,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 5000
params =  {"categories": "accommodation.hotel", 
    "limit": 1,  
    "apiKey": geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    

    # Make and API request using the params dictionary
    name_address =  response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vorgashor - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
mejit - nearest hotel: No hotel found
tongren - nearest hotel: No hotel found
raja - nearest hotel: No hotel found
nuuk - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
derazhnya - nearest hotel: No hotel found
vuktyl - nearest hotel: No hotel found
flying fish cove - nearest hotel: No hotel found
quellon - nearest hotel: No hotel found
kannus - nearest hotel: No hotel found
saint-francois - nearest hotel: No hotel found
fort mcmurray - nearest hotel: No hotel found
natchez - nearest hotel: No hotel found
bonthe - nearest hotel: No hotel found
salinopolis - nearest hotel: No hotel found
pimentel - nearest hotel: No hotel found
casper - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
43,vorgashor,RU,67.5833,63.9500,100,No hotel found
66,badger,US,64.8000,-147.5333,88,No hotel found
102,port mathurin,MU,-19.6833,63.4167,87,No hotel found
120,mejit,MH,10.2753,170.8646,74,No hotel found
130,tongren,CN,27.7172,109.1853,91,No hotel found
138,raja,SS,8.4596,25.6780,21,No hotel found
146,nuuk,GL,64.1835,-51.7216,63,No hotel found
160,margaret river,AU,-33.9500,115.0667,84,No hotel found
170,derazhnya,UA,49.2692,27.4338,88,No hotel found
176,vuktyl,RU,63.8567,57.3094,100,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",  
    "Lat",  
    geo=True,  
    tiles="OSM",  
    size=100,  
    color="City",  
    hover_cols=["City", "Country", "Hotel Name"],  
    title="Hotels in Selected Cities"
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)